In [1]:
#-------------------------------------------------------------------------------

# Name:        Object bounding box label tool

# Purpose:     Label object bboxes for ImageNet Detection data

# Author:      Qiushi

# Created:     06/06/2014



#

#-------------------------------------------------------------------------------

from __future__ import division
from tkinter import *
#import tkMessageBox
from PIL import Image, ImageTk
from tkinter import ttk
import os
import glob
import random

# colors for the bboxes
COLORS = ['red', 'blue', 'olive', 'teal', 'cyan', 'green', 'black']
# image sizes for the examples
SIZE = 256, 256

class LabelTool():
    def __init__(self, master):
        # set up the main frame
        self.parent = master
        self.parent.title("LabelTool")
        self.frame = Frame(self.parent)
        self.frame.pack(fill=BOTH, expand=1)
        self.parent.resizable(width = FALSE, height = FALSE)

        # initialize global state
        self.imageDir = ''
        self.imageList= []
        self.egDir = ''
        self.egList = []
        self.outDir = ''
        self.cur = 0
        self.total = 0
        self.category = 0
        self.imagename = ''
        self.labelfilename = ''
        self.tkimg = None
        self.currentLabelclass = ''
        self.cla_can_temp = []
        self.classcandidate_filename = 'class.txt'

        # initialize mouse state
        self.STATE = {}
        self.STATE['click'] = 0
        self.STATE['x'], self.STATE['y'] = 0, 0

        # reference to bbox
        self.bboxIdList = []
        self.bboxId = None
        self.bboxList = []
        self.hl = None
        self.vl = None

        # ----------------- GUI stuff ---------------------
        # dir entry & load
        self.label = Label(self.frame, text = "Image Dir:")
        self.label.grid(row = 0, column = 0, sticky = E)
        self.entry = Entry(self.frame)
        self.entry.grid(row = 0, column = 1, sticky = W+E)
        self.ldBtn = Button(self.frame, text = "Load", command = self.loadDir)
        self.ldBtn.grid(row = 0, column = 2,sticky = W+E)

        # main panel for labeling
        self.mainPanel = Canvas(self.frame, cursor='tcross')
        self.mainPanel.bind("<Button-1>", self.mouseClick)
        self.mainPanel.bind("<Motion>", self.mouseMove)
        self.parent.bind("<Escape>", self.cancelBBox)  # press <Espace> to cancel current bbox
        self.parent.bind("s", self.cancelBBox)
        self.parent.bind("a", self.prevImage) # press 'a' to go backforward
        self.parent.bind("d", self.nextImage) # press 'd' to go forward
        self.mainPanel.grid(row = 1, column = 1, rowspan = 4, sticky = W+N)

        # choose class
        self.classname = StringVar()
        self.classcandidate = ttk.Combobox(self.frame,state='readonly',textvariable=self.classname)
        self.classcandidate.bind("<FocusIn>", self.setClass)
        self.classcandidate.grid(row=1,column=2)
        if os.path.exists(self.classcandidate_filename):
            with open(self.classcandidate_filename) as cf:
                for line in cf.readlines():
                    #print(line)
                    self.cla_can_temp.append(line.strip('\n'))
        #print self.cla_can_temp
        self.classcandidate['values'] = self.cla_can_temp
        #self.classcandidate.current(0)
        self.currentLabelclass = self.classcandidate.get() #init
        self.currentLabelclassidx = self.classcandidate.current() #init
        self.classname.trace("w", self.setClass)
        self.btnclass = Button(self.frame, text = 'ComfirmClass', command = self.setClass)
        self.btnclass.grid(row=2,column=2,sticky = W+E)

        # showing bbox info & delete bbox
        self.lb1 = Label(self.frame, text = 'Bounding boxes:')
        self.lb1.grid(row = 3, column = 2,  sticky = W+N)
        self.listbox = Listbox(self.frame, width = 22, height = 12)
        self.listbox.grid(row = 4, column = 2, sticky = N+S)
        self.btnDel = Button(self.frame, text = 'Delete', command = self.delBBox)
        self.btnDel.grid(row = 5, column = 2, sticky = W+E+N)
        self.btnClear = Button(self.frame, text = 'ClearAll', command = self.clearBBox)
        self.btnClear.grid(row = 6, column = 2, sticky = W+E+N)

        # control panel for image navigation
        self.ctrPanel = Frame(self.frame)
        self.ctrPanel.grid(row = 7, column = 1, columnspan = 2, sticky = W+E)
        self.prevBtn = Button(self.ctrPanel, text='<< Prev', width = 10, command = self.prevImage)
        self.prevBtn.pack(side = LEFT, padx = 5, pady = 3)
        self.nextBtn = Button(self.ctrPanel, text='Next >>', width = 10, command = self.nextImage)
        self.nextBtn.pack(side = LEFT, padx = 5, pady = 3)
        self.progLabel = Label(self.ctrPanel, text = "Progress:     /    ")
        self.progLabel.pack(side = LEFT, padx = 5)
        self.tmpLabel = Label(self.ctrPanel, text = "Go to Image No.")
        self.tmpLabel.pack(side = LEFT, padx = 5)
        self.idxEntry = Entry(self.ctrPanel, width = 5)
        self.idxEntry.pack(side = LEFT)
        self.goBtn = Button(self.ctrPanel, text = 'Go', command = self.gotoImage)
        self.goBtn.pack(side = LEFT)


        # example pannel for illustration
        self.egPanel = Frame(self.frame, border = 10)
        self.egPanel.grid(row = 1, column = 0, rowspan = 5, sticky = N)
        self.tmpLabel2 = Label(self.egPanel, text = "Examples:")
        self.tmpLabel2.pack(side = TOP, pady = 5)
        self.egLabels = []
        for i in range(3):
            self.egLabels.append(Label(self.egPanel))
            self.egLabels[-1].pack(side = TOP)

        # display mouse position
        self.disp = Label(self.ctrPanel, text='')
        self.disp.pack(side = RIGHT)

        self.frame.columnconfigure(1, weight = 1)
        self.frame.rowconfigure(4, weight = 1)

        # for debugging
##        self.setImage()
##        self.loadDir()

    def loadDir(self, dbg = False):
        print('debug = ', dbg)
        if not dbg:
            s = self.entry.get()
            self.parent.focus()
            self.category = int(s)
        else:
            s = r'../darknet/data/'
##        if not os.path.isdir(s):
##            tkMessageBox.showerror("Error!", message = "The specified dir doesn't exist!")
##            return
        # get image list
        self.imageDir = os.path.join(os.getcwd(),'Images', '%03d' %(self.category))
        print(self.imageDir)
        #print self.category
        self.imageList = glob.glob(os.path.join(self.imageDir, '*.JPG'))
        print(self.imageList)
        if len(self.imageList) == 0:
            print('No .JPG images found in the specified dir!')
            return

        # default to the 1st image in the collection
        self.cur = 1

        self.total = len(self.imageList)



         # set up output dir

        self.outDir = os.path.join(r'./Labels', '%03d' %(self.category))

        if not os.path.exists(self.outDir):

            os.mkdir(self.outDir)



        # load example bboxes

        self.egDir = os.path.join(r'./Examples', '%03d' %(self.category))

        #self.egDir = os.path.join(r'./Examples/demo')

        print(os.path.exists(self.egDir))

        if not os.path.exists(self.egDir):

            return

        filelist = glob.glob(os.path.join(self.egDir, '*.JPG'))

        self.tmp = []

        self.egList = []

        random.shuffle(filelist)

        for (i, f) in enumerate(filelist):

            if i == 3:

                break

            im = Image.open(f)

            r = min(SIZE[0] / im.size[0], SIZE[1] / im.size[1])

            new_size = int(r * im.size[0]), int(r * im.size[1])

            self.tmp.append(im.resize(new_size, Image.ANTIALIAS))

            self.egList.append(ImageTk.PhotoImage(self.tmp[-1]))

            self.egLabels[i].config(image = self.egList[-1], width = SIZE[0], height = SIZE[1])



        self.loadImage()

        print('%d images loaded from %s' %(self.total, s))



    def loadImage(self):

        # load image

        imagepath = self.imageList[self.cur - 1]

        tempimg = Image.open(imagepath)
        basewidth = 400
        wpercent = (basewidth / float(tempimg.size[0]))
        hsize = int((float(tempimg.size[1]) * float(wpercent)))
        self.img = tempimg.resize((basewidth, hsize), Image.ANTIALIAS)


        self.tkimg = ImageTk.PhotoImage(self.img)

        self.mainPanel.config(width = max(self.tkimg.width(), 400), height = max(self.tkimg.height(), 400))

        self.mainPanel.create_image(0, 0, image = self.tkimg, anchor=NW)

        self.progLabel.config(text = "%04d/%04d" %(self.cur, self.total))



        # load labels

        self.clearBBox()

        self.imagename = os.path.split(imagepath)[-1].split('.')[0]

        labelname = self.imagename + '.txt'

        self.labelfilename = os.path.join(self.outDir, labelname)

        #bbox_cnt = 0

        if os.path.exists(self.labelfilename):

            with open(self.labelfilename) as f:

                for (i, line) in enumerate(f):

                    #if i == 0:

                        #bbox_cnt = int(line.strip())

                        #continue
                    
                    # tmp = [int(t.strip()) for t in line.split()]

                    tmppc = line.split()
                    tmp = self.unconvert(tuple(tmppc))
                    #print tmp

                    self.bboxList.append(tuple(tmppc))

                    tmpId = self.mainPanel.create_rectangle(int(tmp[0]), int(tmp[1]), \

                                                            int(tmp[2]), int(tmp[3]), \

                                                            width = 2, \

                                                            outline = COLORS[(len(self.bboxList)-1) % len(COLORS)])

                    #print(tmpId

                    self.bboxIdList.append(tmpId)

                    self.listbox.insert(END, '%s : (%d, %d) -> (%d, %d)' %(tmp[4],int(tmp[0]), int(tmp[1]), \

                                                            int(tmp[2]), int(tmp[3])))

                    self.listbox.itemconfig(len(self.bboxIdList) - 1, fg = COLORS[(len(self.bboxIdList) - 1) % len(COLORS)])



    def saveImage(self):
        print("size", self.img.size)

        with open(self.labelfilename, 'w') as f:

            #f.write('%d\n' %len(self.bboxList))

            for bbox in self.bboxList:
                print(bbox)
                #f.write(' '.join(map(str, bbox)) + '\n')
                f.write(' '.join(map(str, bbox)) + '\n')

        print('Image No. %d saved' %(self.cur))





    def mouseClick(self, event):

        if self.STATE['click'] == 0:

            self.STATE['x'], self.STATE['y'] = event.x, event.y

        else:

            x1, x2 = min(self.STATE['x'], event.x), max(self.STATE['x'], event.x)

            y1, y2 = min(self.STATE['y'], event.y), max(self.STATE['y'], event.y)

            self.bboxList.append(self.convert((x1, y1, x2, y2, self.currentLabelclassidx)))

            self.bboxIdList.append(self.bboxId)

            self.bboxId = None

            self.listbox.insert(END, '%s : (%d, %d) -> (%d, %d)' %(self.currentLabelclass,x1, y1, x2, y2))

            self.listbox.itemconfig(len(self.bboxIdList) - 1, fg = COLORS[(len(self.bboxIdList) - 1) % len(COLORS)])

        self.STATE['click'] = 1 - self.STATE['click']



    def mouseMove(self, event):

        self.disp.config(text = 'x: %d, y: %d' %(event.x, event.y))

        if self.tkimg:

            if self.hl:

                self.mainPanel.delete(self.hl)

            self.hl = self.mainPanel.create_line(0, event.y, self.tkimg.width(), event.y, width = 2)

            if self.vl:

                self.mainPanel.delete(self.vl)

            self.vl = self.mainPanel.create_line(event.x, 0, event.x, self.tkimg.height(), width = 2)

        if 1 == self.STATE['click']:

            if self.bboxId:

                self.mainPanel.delete(self.bboxId)

            self.bboxId = self.mainPanel.create_rectangle(self.STATE['x'], self.STATE['y'], \

                                                            event.x, event.y, \

                                                            width = 2, \

                                                            outline = COLORS[len(self.bboxList) % len(COLORS)])



    def cancelBBox(self, event):

        if 1 == self.STATE['click']:

            if self.bboxId:

                self.mainPanel.delete(self.bboxId)

                self.bboxId = None

                self.STATE['click'] = 0



    def delBBox(self):

        sel = self.listbox.curselection()

        if len(sel) != 1 :

            return

        idx = int(sel[0])

        self.mainPanel.delete(self.bboxIdList[idx])

        self.bboxIdList.pop(idx)

        self.bboxList.pop(idx)

        self.listbox.delete(idx)



    def clearBBox(self):

        for idx in range(len(self.bboxIdList)):

            self.mainPanel.delete(self.bboxIdList[idx])

        self.listbox.delete(0, len(self.bboxList))

        self.bboxIdList = []

        self.bboxList = []



    def prevImage(self, event = None):

        self.saveImage()

        if self.cur > 1:

            self.cur -= 1

            self.loadImage()



    def nextImage(self, event = None):

        self.saveImage()

        if self.cur < self.total:

            self.cur += 1

            self.loadImage()



    def gotoImage(self):

        idx = int(self.idxEntry.get())

        if 1 <= idx and idx <= self.total:

            self.saveImage()

            self.cur = idx

            self.loadImage()



    def setClass(self, a, b, c):

        self.currentLabelclass = self.classcandidate.get()
        self.currentLabelclassidx = int(self.classcandidate.current())
        print('set label class to :',self.currentLabelclass, self.currentLabelclassidx)

    def convert(self, box):
        print("size", self.img.size)
        print("box", box)
        dw = 1./self.img.size[0]
        dh = 1./self.img.size[1]
        x1 = int(box[0])
        y1 = int(box[1])
        x2 = int(box[2])
        y2 = int(box[3])
        x = (x1 + x2)/2.0
        y = (y1 + y2)/2.0
        w = x2 - x1
        h = y2 - y1
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
        return (box[4], x, y, w, h)

    def unconvert(self, box):
        dw = self.img.size[0]
        dh = self.img.size[1]
        cat = int(box[0])
        mix = float(box[1])
        miy = float(box[2])
        w = float(box[3])
        h = float(box[4])
        x1 = (mix - w/2.0)*dw
        x2 = (mix + w/2.0)*dw
        y1 = (miy - h/2.0)*dh
        y2 = (miy + h/2.0)*dh
        return (int(x1) ,int(y1) ,int(x2) ,int(y2), self.classcandidate['values'][cat])

##    def setImage(self, imagepath = r'test2.png'):

##        self.img = Image.open(imagepath)

##        self.tkimg = ImageTk.PhotoImage(self.img)

##        self.mainPanel.config(width = self.tkimg.width())

##        self.mainPanel.config(height = self.tkimg.height())

##        self.mainPanel.create_image(0, 0, image = self.tkimg, anchor=NW)



if __name__ == '__main__':

    root = Tk()

    tool = LabelTool(root)

    root.resizable(width =  True, height = True)

    root.mainloop()

debug =  False
C:\Users\erik\Documents\GitHub\BBox-Label-Tool\Images\001
['C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010692.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010693.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010694.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010695.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010696.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010697.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010698.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010703.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010704.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010705.JPG', 'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool\\Images\\001\\P1010706.JPG', 'C:\\Users\\erik\\Docume

200 images loaded from 001
size (400, 300)
('0', '0.10250000000000001', '0.6966666666666667', '0.185', '0.62')
('0', '0.81', '0.6900000000000001', '0.215', '0.62')
('1', '0.445', '0.6733333333333333', '0.3', '0.64')
('2', '0.8225', '0.48333333333333334', '0.125', '0.19333333333333336')
('2', '0.4825', '0.465', '0.135', '0.21000000000000002')
('2', '0.10875', '0.49833333333333335', '0.1625', '0.23')
('7', '0.095', '0.7766666666666667', '0.145', '0.30000000000000004')
('7', '0.41375', '0.7883333333333333', '0.2325', '0.33')
('7', '0.73625', '0.7983333333333333', '0.0525', '0.4166666666666667')
('3', '0.45625', '0.77', '0.1975', '0.44')
('3', '0.82125', '0.75', '0.1375', '0.30000000000000004')
('3', '0.06875', '0.8016666666666667', '0.0975', '0.36333333333333334')
Image No. 1 saved
size (400, 300)
('3', '0.5750000000000001', '0.67', '0.21', '0.4866666666666667')
('3', '0.7825', '0.5933333333333334', '0.15', '0.3066666666666667')
('3', '0.04625', '0.6566666666666667', '0.0725', '0.39333333

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\apps\Anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
TypeError: setClass() missing 2 required positional arguments: 'b' and 'c'


set label class to : Rleg 5


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\apps\Anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
TypeError: setClass() missing 2 required positional arguments: 'b' and 'c'


...

size (400, 300)
('1', '0.7925', '0.6233333333333334', '0.41000000000000003', '0.23333333333333334')
('1', '0.8075', '0.26666666666666666', '0.39', '0.28')
('2', '0.6375000000000001', '0.5800000000000001', '0.095', '0.08666666666666667')
('6', '0.755', '0.6783333333333333', '0.195', '0.11')
('5', '0.9', '0.6483333333333334', '0.20500000000000002', '0.08333333333333334')
('4', '0.89625', '0.5766666666666667', '0.20750000000000002', '0.060000000000000005')
('6', '0.7625000000000001', '0.5666666666666667', '0.16', '0.10666666666666667')
('3', '0.73', '0.6133333333333334', '0.145', '0.16')
Image No. 35 saved
size (400, 300)
Image No. 34 saved
size (400, 300)
('1', '0.65125', '0.5833333333333334', '0.4175', '0.8200000000000001')
('0', '0.3075', '0.66', '0.365', '0.6533333333333333')
('2', '0.305', '0.6666666666666667', '0.355', '0.64')
('2', '0.65125', '0.5783333333333334', '0.4025', '0.8033333333333333')
Image No. 33 saved
size (400, 300)
('1', '0.57625', '0.535', '0.8425', '0.9166666666666

size (400, 300)
('0', '0.485', '0.53', '0.925', '0.5733333333333334')
('2', '0.09875', '0.5566666666666668', '0.1475', '0.19333333333333336')
('7', '0.20375000000000001', '0.4366666666666667', '0.1675', '0.39333333333333337')
('5', '0.665', '0.7000000000000001', '0.55', '0.2066666666666667')
('4', '0.66125', '0.49833333333333335', '0.47750000000000004', '0.21000000000000002')
('3', '0.28500000000000003', '0.6233333333333334', '0.23500000000000001', '0.25333333333333335')
Image No. 45 saved
size (400, 300)
('0', '0.5175', '0.315', '0.9500000000000001', '0.5033333333333334')
('2', '0.14', '0.36500000000000005', '0.18', '0.21666666666666667')
('3', '0.44375000000000003', '0.32833333333333337', '0.2675', '0.2966666666666667')
('5', '0.78', '0.4033333333333334', '0.41500000000000004', '0.21333333333333335')
('4', '0.77625', '0.18000000000000002', '0.4325', '0.2')
('6', '0.4225', '0.1716666666666667', '0.38', '0.1366666666666667')
('7', '0.505', '0.48000000000000004', '0.505', '0.12000000000

('6', '0.53125', '0.7000000000000001', '0.0925', '0.26666666666666666')
('6', '0.80875', '0.67', '0.17250000000000001', '0.34')
Image No. 25 saved
size (400, 300)
('0', '0.42', '0.6783333333333333', '0.275', '0.48333333333333334')
('0', '0.7175', '0.5416666666666667', '0.225', '0.3433333333333334')
('1', '0.17500000000000002', '0.6566666666666667', '0.335', '0.66')
('1', '0.9175', '0.6166666666666667', '0.155', '0.42000000000000004')
('2', '0.23375', '0.4733333333333334', '0.1875', '0.26')
('2', '0.42125', '0.5333333333333333', '0.1075', '0.16')
('2', '0.6950000000000001', '0.4816666666666667', '0.11', '0.18333333333333335')
('2', '0.93125', '0.5116666666666667', '0.1275', '0.20333333333333334')
('7', '0.17', '0.785', '0.325', '0.38333333333333336')
('7', '0.4225', '0.7100000000000001', '0.25', '0.23333333333333334')
Image No. 24 saved
size (400, 300)
('0', '0.34', '0.675', '0.455', '0.65')
('0', '0.65375', '0.6266666666666667', '0.6525', '0.7333333333333334')
('2', '0.48875', '0.44333

size (400, 300)
('1', '0.43', '0.4683333333333334', '0.18', '0.45')
('1', '0.075', '0.3866666666666667', '0.14', '0.3866666666666667')
('0', '0.11625', '0.5133333333333334', '0.2025', '0.44')
('2', '0.41000000000000003', '0.29333333333333333', '0.05', '0.09333333333333334')
('2', '0.11375', '0.24666666666666667', '0.0675', '0.10666666666666667')
('2', '0.18625', '0.34500000000000003', '0.0675', '0.09666666666666668')
('4', '0.4275', '0.5700000000000001', '0.04', '0.22')
('5', '0.3775', '0.5633333333333334', '0.06', '0.22')
('5', '0.075', '0.5766666666666667', '0.095', '0.2866666666666667')
('7', '0.14125000000000001', '0.46166666666666667', '0.0975', '0.19666666666666668')
('7', '0.365', '0.4266666666666667', '0.045', '0.2')
('7', '0.06', '0.41000000000000003', '0.1', '0.28')
('6', '0.47750000000000004', '0.35500000000000004', '0.09', '0.06333333333333334')
Image No. 10 saved
size (400, 300)
('1', '0.41125', '0.5900000000000001', '0.11750000000000001', '0.46')
('1', '0.15', '0.60833333

size (400, 300)
('0', '0.10250000000000001', '0.6966666666666667', '0.185', '0.62')
('0', '0.81', '0.6900000000000001', '0.215', '0.62')
('1', '0.445', '0.6733333333333333', '0.3', '0.64')
('2', '0.8225', '0.48333333333333334', '0.125', '0.19333333333333336')
('2', '0.4825', '0.465', '0.135', '0.21000000000000002')
('2', '0.10875', '0.49833333333333335', '0.1625', '0.23')
('7', '0.095', '0.7766666666666667', '0.145', '0.30000000000000004')
('7', '0.41375', '0.7883333333333333', '0.2325', '0.33')
('7', '0.73625', '0.7983333333333333', '0.0525', '0.4166666666666667')
('3', '0.45625', '0.77', '0.1975', '0.44')
('3', '0.82125', '0.75', '0.1375', '0.30000000000000004')
('3', '0.06875', '0.8016666666666667', '0.0975', '0.36333333333333334')
Image No. 1 saved
size (400, 300)
('0', '0.10250000000000001', '0.6966666666666667', '0.185', '0.62')
('0', '0.81', '0.6900000000000001', '0.215', '0.62')
('1', '0.445', '0.6733333333333333', '0.3', '0.64')
('2', '0.8225', '0.48333333333333334', '0.125', 

In [ ]:
os.getcwd()

In [ ]:
imagepath = "C:/Users/erik/Documents/GitHub/BBox-Label-Tool/Images/001/P1090459.JpG"
tempimg = Image.open(imagepath)
if tempimg.width > 400 or tempimg.height > 400:
    ratio = max(tempimg.width, tempimg.width)/400
else:
    ratio = 1
resized = ratio*tempimg.width, ratio*tempimg.height
#img = tempimg.thumbnail(resized,Image.ANTIALIAS)
tkimg = ImageTk.PhotoImage(tempimg)
print(tkimg.size)
      

In [ ]:
def convert(size, box):
        print("size", size)
        print("box", box)
        dw = 1./size[0]
        dh = 1./size[1]
        x = (box[0] + box[2])/2.0
        y = (box[1] + box[3])/2.0
        w = box[2] - box[0]
        h = box[3] - box[1]
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
        return (box[4], x, y, w, h)
    
def unconvert(size, box):
        dw = size[0]
        dh = size[1]
        x1 = (box[1] - box[3]/2.0)*dw
        x2 = (box[1] + box[3]/2.0)*dw
        y1 = (box[2] - box[4]/2.0)*dh
        y2 = (box[2] + box[4]/2.0)*dh
        return (x1 ,x2 ,y1 ,y2, box[0])

unconvert((400,200),convert((400,200),(100,200,50,100,'k')))

In [47]:
convert((400,300),(0,400,0,300,'human'))

('human', 0.5, 0.5, 1.0, 1.0)

In [50]:
unconvert((400,300),('human',0.5,0.5,1,1))

(0.0, 400.0, 0.0, 300.0, 'human')

In [59]:
convert((400, 300), (6, 4, 400, 298, ''))

size (400, 300)
box (6, 4, 400, 298, '')


('', 0.5075000000000001, 0.5033333333333334, 0.985, 0.9800000000000001)

In [69]:
tup = (1,2,5,"we")

In [75]:
tup.index(1)

0

In [81]:
classcandidate = ttk.Combobox(Frame(),state='readonly',textvariable="h")

In [3]:
import random
imagepath = "C:/Users/erik/Documents/GitHub/BBox-Label-Tool/Images/001"
labeltrainfilename = "C:/Users/erik/Documents/GitHub/darknet/train.txt"
labeltestfilename = "C:/Users/erik/Documents/GitHub/darknet/test.txt"
imagelist = [label for label in os.listdir(imagepath) if label.split('.')[1] == 'JPG']
idxs = random.sample(range(len(imagelist)),int(len(imagelist)*0.9))
with open(labeltrainfilename, 'w') as f, open(labeltestfilename, 'w') as g:

        for i,label in enumerate(imagelist):
            #label = label.split('.')[0]
            if i in idxs:
                f.write("data/001/{0}\n".format(label))
            else:
                g.write("data/001/{0}\n".format(label))




In [4]:
os.getcwd()

'C:\\Users\\erik\\Documents\\GitHub\\BBox-Label-Tool'